## Imports

In [1]:
import random                  
import math
from fractions import Fraction
import numpy as np
import pandas as pd
from music21 import chord, corpus, environment, meter, midi, note, stream, tempo, converter

In [2]:
def play(score):
    #Shortcut to play a stream
    midi.realtime.StreamPlayer(score).play()

## Setup

In [3]:
score = converter.parse('score.mxl')
r_hand = score.parts[0]
l_hand = score.parts[1]
r_measures = r_hand.getElementsByClass('Measure')
l_measures = l_hand.getElementsByClass('Measure')

piece_end = r_measures[-1].offset + 4

## Pitch degeneration

In [4]:
def shift_pitch_fun(delta_fun, start_offset=0, end_offset=piece_end, include_end=False):
    notes = score.flat.getElementsByOffset(start_offset, end_offset, includeEndBoundary=include_end, classList=[note.Note])
    for n in notes:
        cents = n.pitch.microtone.cents
        n.pitch.microtone = cents + delta_fun(n.offset)
    
    chords = score.flat.getElementsByOffset(start_offset, end_offset, includeEndBoundary=include_end, classList=[chord.Chord])
    for c in chords:
        for n in c.notes:
            cents = n.pitch.microtone.cents
            n.pitch.microtone = cents + delta_fun(c.offset)

def shift_pitch(delta, start_offset=0, end_offset=piece_end, include_end=False):
    shift_pitch_fun(lambda x: delta, start_offset, end_offset, include_end)

In [5]:
shift_pitch(25, 4*11)
shift_pitch(50, 4*11*2)
shift_pitch(-75, 4*11*3)

def gradual_shift(offset):
    progression = offset/piece_end
    max_delta = 100*(progression**1.5)
    return random.uniform(-max_delta, max_delta)
    
shift_pitch_fun(gradual_shift)

# Rhythm

In [6]:
r_hand_flat = r_hand.flat
l_hand_flat = l_hand.flat

In [7]:
xs = [1 - random.uniform(0, 1)**1.3 for x in range(10)]
xs = sorted(xs)[::-1]

last_x = 10000
for x in xs:
    #off = math.floor(random.uniform(0.25, 0.25 + 0.55*x)*4)/4
    off = random.uniform(0.05, 0.1 + 0.25*x)
    x = 4*11 + (piece_end - 4*11 - 4*5)*x
    if (x + off*2 > last_x):
        continue
    print(off)
    print(x)
    r_hand_flat.shiftElements(off, x)
    l_hand_flat.shiftElements(off, x)
    if (random.uniform(0, 1) > 1/2):
        r_hand_flat.getElementsByOffset(0, x, classList=[note.Note])[-1].quarterLength += off
        l_hand_flat.getElementsByOffset(0, x, classList=[chord.Chord])[-1].quarterLength += off
    last_x = x

0.18941275155523635
175.80523460501405
0.30915016757341185
174.74541616263235
0.10321956038726209
172.65560377071532
0.09285331362021992
156.16539682275555
0.20280357536363158
122.99779159694897
0.12799225830285402
103.23829665129666
0.12012168705654852
98.25249827621201
0.13607819794016873
72.02327121088999
0.07539868654079544
67.8440305336365


In [8]:
new_stream = stream.Stream()
new_stream.append(r_hand_flat)
new_stream.append(l_hand_flat)

## Export to MIDI

In [9]:
new_stream.write('midi', fp='out.mid')

'out.mid'

In [10]:
new_stream.show('text')

{0.0} <music21.stream.PartStaff P1-Staff1_flat>
    {0.0} <music21.instrument.Instrument 'P1: Piano: Piano'>
    {0.0} <music21.layout.SystemLayout>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.key.KeySignature of no sharps or flats>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Rest rest>
    {1.0} <music21.note.Note G#>
    {2.0} <music21.note.Note F#>
    {2.75} <music21.note.Note C>
    {3.0} <music21.note.Note C>
    {3.6667} <music21.note.Note D->
    {4.0} <music21.note.Note D->
    {6.0} <music21.note.Note G#>
    {7.0} <music21.note.Note C>
    {8.0} <music21.note.Note D->
    {9.0} <music21.note.Note G#>
    {10.0} <music21.note.Note G#>
    {11.0} <music21.note.Note E>
    {12.0} <music21.note.Note F#>
    {14.0} <music21.note.Note E>
    {14.5} <music21.note.Rest rest>
    {15.0} <music21.note.Note E>
    {16.0} <music21.note.Note F#>
    {18.0} <music21.note.Note A>
    {20.0} <music21.note.Note D->
    {22.0} <music21.note.Note B>
    {23.0